In [2]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

import pyqg

# Run barotropic model following the example but with a mild meridional topographic gradient for testing

Following the pyqg example for McWilliams 1984 freely-evolving 2D turbulence ($R_d = 0, \beta = 0$) experiments on a $2 \pi \times 2 \pi$ doubly-periodic box of $H = 1$, but now with a linear slope in meridional topography (i.e., isomorphic to having $\beta \neq 0$). See https://pyqg.readthedocs.io/en/latest/examples/barotropic.html.

In [28]:
# Grid, dimensions
L = 2 * np.pi
Ld = None
nx = 256.

# Height and topography
H = 1.
hy = 0.#1e-5

# Plantary stuff
f0 = 1e-4
beta = 0.
rek = 0.

# Time
tmax = 40
dt = 0.001
taveint = 1


# Create the model object
# m = pyqg.BTModel(L = L, rd = Ld, nx = nx,
#                  f0 = f0, beta = beta, rek = rek,
#                  H = H, hy = hy,
#                  tmax = tmax, dt = dt, taveint = taveint)



m = pyqg.BTModel(L=2.*np.pi, nx=256,
                 beta=0., H=1., rek=0., rd=None,
                 tmax=40, dt=0.001, twrite = 1)

# # generate McWilliams 84 IC condition

# fk = m.wv != 0
# ckappa = np.zeros_like(m.wv2)
# ckappa[fk] = np.sqrt( m.wv2[fk]*(1. + (m.wv2[fk]/36.)**2) )**-1

# nhx,nhy = m.wv2.shape

# Pi_hat = np.random.randn(nhx,nhy)*ckappa +1j*np.random.randn(nhx,nhy)*ckappa

# Pi = m.ifft( Pi_hat[np.newaxis,:,:] )
# Pi = Pi - Pi.mean()
# Pi_hat = m.fft( Pi )
# KEaux = m.spec_var( m.wv*Pi_hat )

# pih = ( Pi_hat/np.sqrt(KEaux) )
# qih = -m.wv2*pih
# qi = m.ifft(qih)

# # initialize the model with that initial condition
# m.set_q(qi)

# # define a quick function for plotting and visualize the initial condition
# def plot_q(m, qmax=40):
#     fig, ax = plt.subplots(figsize=(10,10))
#     pc = ax.pcolormesh(m.x,m.y,m.q.squeeze(), cmap='RdBu_r')
#     pc.set_clim([-qmax, qmax])
#     ax.set_xlim([0, 2*np.pi])
#     ax.set_ylim([0, 2*np.pi]);
#     ax.set_aspect(1)
#     plt.colorbar(pc)
#     plt.title('Time = %g' % m.t)
#     plt.show()

# plot_q(m)

INFO:  Logger initialized


In [42]:
m.run()

INFO: Step: 14, Time: 1.40e-02, KE: nan, CFL:  nan
ERROR: CFL condition violated


AssertionError: None

In [23]:
for _ in m.run_with_snapshots(tsnapstart=0, tsnapint=1):
    plot_q(m)

INFO: Step: 15, Time: 1.50e-02, KE: nan, CFL:  nan
ERROR: CFL condition violated


AssertionError: None

# Run barotropic model on a beta-plane with a mild meridional topographic gradient for testing

In [45]:
# Grid, dimensions
nx = 256.
L = 1000e3
Ld = None
H = 5000

# Planetary stuff
f0 = 1e-4
omega = 7.2921159e-5
a = 6.371e6
lat = np.arcsin(f0 / (2 * omega))
beta = 2 * omega / a * np.cos(lat)

rek = 1 / (25 * 24 * 60 * 60) # linear bottom drag coeff.  [s^-1]

# Topography
hy = 0. #1e-6
hx = 0.

# Time defaults (the default settings chosen by pyqg)
year = 24 * 60 * 60 * 365.  

# tmax = 1.577e8        # Run for 5 years [s]
# tavestart = 3.154e6   # Start averaring after 1 year [s]
# taveint = 86400.      # Daily averages [s]
# dt = 3600. / 100           # Hourly time step [s]

tmax = 10 * year
tavestart = 5 * year
twrite = 1

dt = 7200. / 10


# Initialize model object
m = pyqg.QGModel(L = L, rd = Ld, nx = nx,
                f0 = f0, beta = beta, rek = rek,
                 U
                H1 = H, delta = 1., hy = hy,
                tmax = tmax, tavestart = tavestart, dt = dt, twrite = twrite)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (638842169.py, line 38)

In [18]:
for _ in m.run_with_snapshots(tsnapstart = 0, tsnapint = dt):
    plot_q(m)

INFO: Step: 4000, Time: 4.00e+00, KE: nan, CFL:  nan
ERROR: CFL condition violated


AssertionError: None

# Run 2 layer QG model that was working in the default library configuration

In [3]:
# Run 2 layer QG for 15 years

L =  1000e3     # length scale of box    [m]
Ld = 50e3       # deformation scale      [m]
#kd = 1. / Ld    # deformation wavenumber [m^-1]
Nx = 256.       # number of grid points

U1 = 0.01#0.03        # layer 1 background zonal velocity [m/s]
U2 = 0.          # layer 2 background zonal velocity [m/s]
H1 = 500.        # depth of layer 1
delta = 1.       # layer depth ratio, \delta = H_1 / H_2, with total depth H = H_1 + H_2

hy = 0.
hx = 1e-5

rek = 1 / (25 * 24 * 60 * 60) # linear bottom drag coeff.  [s^-1]
f0  = 1e-4                    # coriolis param [s^-1]
beta = 1.5e-11                # planetary vorticity gradient [m^-1 s^-1]

year = 24 * 60 * 60 * 365.    
tmax = 15 * year
Ti = Ld / (abs(U1))   # estimate of most unstable e-folding time scale [s] (see 3-layer example in documentation)
dt = Ti / 200.        # time-step [s]

# Run the model for 15 years, and average over the last 5 years.

m = pyqg.QGModel(nx = Nx, L = L, rd = Ld, U1 = U1, U2 = U2, H1 = H1, delta = delta,
                 hx = hx, hy = hy,
                 f = f0, beta = beta, rek = rek,
                 dt = dt / 2, tmax = tmax, twrite = 1, tavestart = tmax - 5 * year)
# m.run()

INFO:  Logger initialized


In [8]:
dt/2, m.dt

(12500.0, 12500.0)

NOTE: above parameters but with topographic slopes: hy = 0 and hx = 1e-5 violated CFL condition sometime before the 10,000th timestep... But runs past this time with hx = 1e-7.